In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
cols = ["CHFL", "IRIS", "CMBL", "NPERR", "STATR","NENFR","NE3FR", "CS1"]

In [3]:
df = pd.read_csv("../input/original-census/FD_INDCVI_2018.csv", delimiter=";", usecols=cols)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3524: DtypeWarning: Columns (19,22,47,63) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
for i in cols:
    df[i]=df[i].astype("str")

In [5]:
#df[cols].to_csv("CENSUS_SHORT.csv", index=False)

In [6]:
df["count_per_iris"] = df.groupby("IRIS")["NE3FR"].transform("count")

In [7]:
df["DEPCOM"] = df["IRIS"].str[:5]

In [8]:
df_list=[]
cols_f = ["CHFL", "CMBL","CS1", "NPERR", "STATR","NENFR"]
for i in cols_f:
    print(i)
    g_data = df.groupby(["IRIS", i]).count()["NE3FR"].reset_index()
    g_data.rename(columns={"NE3FR": "count_per_cat"},inplace=True)
    g_data["count"] = g_data.groupby("IRIS")["count_per_cat"].transform('sum')
    g_data[f"freq_{i}"] = g_data["count_per_cat"]/g_data["count"]
    g_data[i] = f"{i}_" + g_data[i]
    g_data = g_data[["IRIS", "count", i, f"freq_{i}"]].pivot_table(f"freq_{i}", ['IRIS'], i)
    g_data = g_data.rename_axis(None).rename_axis(None, axis=1)
    df_list.append(g_data)
    

CHFL
CMBL
CS1
NPERR
STATR
NENFR


In [9]:
chfl_cmbl = []
for i in ["1","2"]:
    g_data = df[df["CHFL"]==i].groupby(["IRIS", "CMBL", "count_per_iris"]).count()["NE3FR"].reset_index()
    #g_data["count"] = g_data.groupby("IRIS")["NE3FR"].transform('sum')
    g_data.rename(columns={"NE3FR": "count_per_cat"},inplace=True)
    g_data[f"freq_CHFL{i}_CMBL"] = g_data["count_per_cat"]/g_data["count_per_iris"]
    g_data["CMBL"] = f"CHFL{i}_CMBL_" + g_data["CMBL"]
    g_data.rename(columns={"CMBL": f"CHFL{i}_CMBL"}, inplace=True)
    g_data = g_data[["IRIS", "count_per_iris", f"CHFL{i}_CMBL", f"freq_CHFL{i}_CMBL"]].pivot_table(f"freq_CHFL{i}_CMBL", ['IRIS'], f"CHFL{i}_CMBL")
    g_data = g_data.rename_axis(None).rename_axis(None, axis=1)
    g_data.reset_index(inplace=True)
    chfl_cmbl.append(g_data)

In [10]:
#df.groupby(["IRIS", "CMBL", "count_per_iris"]).count()["NE3FR"].reset_index().sort_values(by="IRIS")

In [11]:
census = pd.concat(df_list, axis=1).reset_index()

In [12]:
census.shape

(16209, 39)

In [13]:
census = census.merge(chfl_cmbl[0], how="left", on="index")
census = census.merge(chfl_cmbl[1], how="left", on="index")

In [14]:
census.shape

(16209, 50)

In [15]:
census.rename(columns={"index":"IRIS"}, inplace=True)

In [16]:
census.to_csv("census_prepared_IRIS_2018.csv", index=False)

In [17]:
df[df["CHFL"]=="1"]["CMBL"].value_counts()

2    1882107
1    1503690
4     351960
3     291290
6     194105
5      26915
Name: CMBL, dtype: int64

In [18]:
df[(df["IRIS"]=="595990901")][["CHFL", "CMBL", "count_per_iris"]]

,CHFL,CMBL,count_per_iris
9667058,1,2,1517
9667103,2,2,1517
9667104,2,2,1517
9667109,2,4,1517
9667112,2,4,1517
...,...,...,...
9703375,X,X,1517
9703376,X,X,1517
9703377,X,X,1517
9703378,X,X,1517


In [19]:
census[["CHFL1_CMBL_1", "CMBL_1", "CHFL_1"]]

,CHFL1_CMBL_1,CMBL_1,CHFL_1
0,0.017456,0.017456,0.071072
1,0.072072,0.072072,0.410210
2,0.024469,0.024469,0.278171
3,0.000555,0.000555,0.013311
4,0.077125,0.077125,0.378615
...,...,...,...
16204,NaN,NaN,NaN
16205,NaN,NaN,NaN
16206,NaN,NaN,NaN
16207,NaN,NaN,NaN


In [20]:
df[df["STATR"]=="1"]["CS1"].value_counts()

5    2195466
4    1945424
6    1501639
3    1380989
Name: CS1, dtype: int64

In [21]:
df[df["STATR"]=="2"]["CS1"].value_counts()

2    509335
3    186660
4    124630
1     90568
Name: CS1, dtype: int64

In [22]:
df[df["STATR"]=="Z"]["CS1"].value_counts()

8    6529418
7    3996851
5     439660
6     368772
4     219066
3      74967
2      38034
1       1792
Name: CS1, dtype: int64

In [23]:
df[df["STATR"]=="Z"]

,CHFL,CMBL,CS1,IRIS,NE3FR,NENFR,NPERR,STATR,count_per_iris,DEPCOM
0,2,4,7,ZZZZZZZZZ,0,0,2,Z,5537759,ZZZZZ
1,2,4,7,ZZZZZZZZZ,0,0,2,Z,5537759,ZZZZZ
7,2,2,8,010040101,Z,Z,1,Z,802,01004
13,2,4,6,010040102,0,1,2,Z,1665,01004
15,1,1,6,010040102,Z,Z,1,Z,1665,01004
...,...,...,...,...,...,...,...,...,...,...
19603266,X,X,8,974220101,Z,Z,Z,Z,1135,97422
19603267,X,X,8,974220101,Z,Z,Z,Z,1135,97422
19603268,X,X,8,974220102,Z,Z,Z,Z,793,97422
19603269,X,X,8,974220102,Z,Z,Z,Z,793,97422


In [24]:
df[df["CHFL"]=="1"]["CMBL"].value_counts()

2    1882107
1    1503690
4     351960
3     291290
6     194105
5      26915
Name: CMBL, dtype: int64